<a href="https://colab.research.google.com/github/kszymon/neural-network/blob/main/05_regression%20/01_housing_prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Problem regresji - prognoza ceny mieszkań

Celem tego notebook'a jest zapoznanie się z problemami regresji. W odróżnieniu od problemów klasyfikacji, gdzie staramy się przyporządkować daną klasę, w modelach regresji przewidujemy ciągłą zmienną docelową.

### Spis treści:
1. [Import bibliotek](#a0)
2. [Załadowanie danych i wstępna eksploracja](#a1)
3. [Podział na zbiór treningowy oraz testowy](#a2)
4. [Standaryzacja danych](#a3)
5. [Budowa modelu](#a4)
6. [Trenowanie sieci](#a5)




### <a name='a0'></a> 1. Import biblotek

In [30]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

np.set_printoptions(precision=12, suppress=True, linewidth=150)
pd.options.display.float_format = '{:.6f}'.format
tf.__version__

'2.18.0'

### <a name='a1'></a> 2. Załadowanie danych i wstępna eksploracja

In [31]:
raw_dataset = pd.read_csv('https://storage.googleapis.com/esmartdata-courses-files/ann-course/housing.csv')
raw_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [32]:
dataset = raw_dataset.copy()
dataset.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.230000,37.880000,41.000000,880.000000,129.000000,322.000000,126.000000,8.325200,452600.000000,NEAR BAY
1,-122.220000,37.860000,21.000000,7099.000000,1106.000000,2401.000000,1138.000000,8.301400,358500.000000,NEAR BAY
2,-122.240000,37.850000,52.000000,1467.000000,190.000000,496.000000,177.000000,7.257400,352100.000000,NEAR BAY
3,-122.250000,37.850000,52.000000,1274.000000,235.000000,558.000000,219.000000,5.643100,341300.000000,NEAR BAY
4,-122.250000,37.850000,52.000000,1627.000000,280.000000,565.000000,259.000000,3.846200,342200.000000,NEAR BAY


In [33]:
dataset.tail()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
20635,-121.090000,39.480000,25.000000,1665.000000,374.000000,845.000000,330.000000,1.560300,78100.000000,INLAND
20636,-121.210000,39.490000,18.000000,697.000000,150.000000,356.000000,114.000000,2.556800,77100.000000,INLAND
20637,-121.220000,39.430000,17.000000,2254.000000,485.000000,1007.000000,433.000000,1.700000,92300.000000,INLAND
20638,-121.320000,39.430000,18.000000,1860.000000,409.000000,741.000000,349.000000,1.867200,84700.000000,INLAND
20639,-121.240000,39.370000,16.000000,2785.000000,616.000000,1387.000000,530.000000,2.388600,89400.000000,INLAND


In [34]:
dataset.isnull().sum() / len(dataset)

,0
longitude,0.000000
latitude,0.000000
housing_median_age,0.000000
total_rooms,0.000000
total_bedrooms,0.010029
population,0.000000
households,0.000000
median_income,0.000000
median_house_value,0.000000
ocean_proximity,0.000000


In [35]:
dataset.dropna(inplace=True)

dataset.isnull().sum() / len(dataset)

,0
longitude,0.000000
latitude,0.000000
housing_median_age,0.000000
total_rooms,0.000000
total_bedrooms,0.000000
population,0.000000
households,0.000000
median_income,0.000000
median_house_value,0.000000
ocean_proximity,0.000000


In [36]:
dataset.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000
mean,-119.570689,35.633221,28.633094,2636.504233,537.870553,1424.946949,499.433465,3.871162,206864.413155
std,2.003578,2.136348,12.591805,2185.269567,421.385070,1133.208490,382.299226,1.899291,115435.667099
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1450.000000,296.000000,787.000000,280.000000,2.563700,119500.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.536500,179700.000000
75%,-118.010000,37.720000,37.000000,3143.000000,647.000000,1722.000000,604.000000,4.744000,264700.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [37]:
dataset.describe(include=['object'])

,ocean_proximity
count,20433
unique,5
top,<1H OCEAN
freq,9034


In [38]:
dataset.ocean_proximity.value_counts()

,count
ocean_proximity,
<1H OCEAN,9034
INLAND,6496
NEAR OCEAN,2628
NEAR BAY,2270
ISLAND,5


In [39]:
px.histogram(dataset, x='median_house_value')

In [40]:
dataset.median_house_value.value_counts()

,count
median_house_value,
500001.000000,958
137500.000000,119
162500.000000,116
112500.000000,103
187500.000000,92
...,...
321700.000000,1
300800.000000,1
393100.000000,1


In [41]:
index_to_drop = dataset[dataset.median_house_value == 500000].index
datset = dataset.drop(index=index_to_drop)
px.histogram(dataset, x='median_house_value')

In [42]:
dataset_dummies = pd.get_dummies(dataset, drop_first=True)
dataset_dummies.head(5)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
0,-122.230000,37.880000,41.000000,880.000000,129.000000,322.000000,126.000000,8.325200,452600.000000,False,False,True,False
1,-122.220000,37.860000,21.000000,7099.000000,1106.000000,2401.000000,1138.000000,8.301400,358500.000000,False,False,True,False
2,-122.240000,37.850000,52.000000,1467.000000,190.000000,496.000000,177.000000,7.257400,352100.000000,False,False,True,False
3,-122.250000,37.850000,52.000000,1274.000000,235.000000,558.000000,219.000000,5.643100,341300.000000,False,False,True,False
4,-122.250000,37.850000,52.000000,1627.000000,280.000000,565.000000,259.000000,3.846200,342200.000000,False,False,True,False


### <a name='a2'></a> 3. Podział na zbiór treningowy oraz testowy

In [43]:
train_dataset = dataset_dummies.sample(frac=0.8, random_state=0)
test_dataset = dataset_dummies.drop(train_dataset.index)

print(f'train_dataset length: {len(train_dataset)}')
print(f'test_dataset lenfth: {len(test_dataset)}')

train_dataset length: 16346
test_dataset lenfth: 4087


In [44]:
train_dataset.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
14185,-117.080000,32.700000,37.000000,2176.000000,418.000000,1301.000000,375.000000,2.875000,98900.000000,False,False,False,True
6125,-117.910000,34.110000,20.000000,3158.000000,684.000000,2396.000000,713.000000,3.525000,153000.000000,False,False,False,False
14095,-117.100000,32.750000,11.000000,2393.000000,726.000000,1905.000000,711.000000,1.344800,91300.000000,False,False,False,True
14359,-117.220000,32.740000,52.000000,1260.000000,202.000000,555.000000,209.000000,7.275800,345200.000000,False,False,False,True
18004,-121.990000,37.290000,32.000000,2930.000000,481.000000,1336.000000,481.000000,6.463100,344100.000000,False,False,False,False


In [45]:
test_dataset.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
10,-122.260000,37.850000,52.000000,2202.000000,434.000000,910.000000,402.000000,3.203100,281500.000000,False,False,True,False
13,-122.260000,37.840000,52.000000,696.000000,191.000000,345.000000,174.000000,2.673600,191300.000000,False,False,True,False
19,-122.270000,37.840000,52.000000,1503.000000,298.000000,690.000000,275.000000,2.603300,162900.000000,False,False,True,False
28,-122.280000,37.840000,50.000000,2082.000000,492.000000,1131.000000,473.000000,1.642400,108900.000000,False,False,True,False
40,-122.260000,37.830000,52.000000,1665.000000,419.000000,946.000000,395.000000,2.097800,155400.000000,False,False,True,False


In [46]:
px.scatter_matrix(train_dataset, dimensions=['median_house_value', 'housing_median_age', 'median_income', 'total_rooms'], color='median_house_value', height=700)

In [47]:
train_stats = train_dataset.describe()
train_stats.pop('median_house_value')
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
longitude,16346.000000,-119.564154,2.002618,-124.350000,-121.790000,-118.490000,-118.000000,-114.470000
latitude,16346.000000,35.630318,2.138574,32.550000,33.930000,34.250000,37.710000,41.950000
housing_median_age,16346.000000,28.664505,12.556764,1.000000,18.000000,29.000000,37.000000,52.000000
total_rooms,16346.000000,2622.235776,2169.548287,11.000000,1448.000000,2119.000000,3120.750000,39320.000000
total_bedrooms,16346.000000,535.281659,418.469078,3.000000,296.000000,432.500000,644.000000,6445.000000
population,16346.000000,1416.087055,1103.842065,3.000000,784.250000,1164.000000,1711.000000,28566.000000
households,16346.000000,496.758167,379.109535,3.000000,280.000000,408.000000,600.000000,6082.000000
median_income,16346.000000,3.869337,1.902228,0.499900,2.555675,3.533200,4.744225,15.000100


In [48]:
train_labels = train_dataset.pop('median_house_value')
test_labels = test_dataset.pop('median_house_value')

### <a name='a3'></a> 4. Standaryzacja danych

In [49]:
continuous_columns = [col for col in train_dataset.columns if not col.startswith('ocean_proximity')]

def norm(x):
    normed = x.copy()

    bool_cols = normed.select_dtypes(include=['bool', 'boolean']).columns
    normed[bool_cols] = normed[bool_cols].astype(int)

    normed[continuous_columns] = (normed[continuous_columns] - train_stats.loc[continuous_columns, 'mean']) / \
                                 train_stats.loc[continuous_columns, 'std']
    return normed

In [50]:
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [51]:
normed_train_data.isnull().sum()

,0
longitude,0
latitude,0
housing_median_age,0
total_rooms,0
total_bedrooms,0
population,0
households,0
median_income,0
ocean_proximity_INLAND,0
ocean_proximity_ISLAND,0


In [52]:
normed_train_data.head(3)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
14185,1.240453,-1.370221,0.663825,-0.205681,-0.280264,-0.104260,-0.321169,-0.522722,0,0,0,1
6125,0.825996,-0.710903,-0.690027,0.246947,0.355387,0.887729,0.570394,-0.181018,0,0,0,0
14095,1.230466,-1.346840,-1.406772,-0.105661,0.455753,0.442919,0.565119,-1.327147,0,0,0,1


In [53]:
normed_test_data.head(3)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
10,-1.346161,1.037926,1.858400,-0.193697,-0.242029,-0.458478,-0.249949,-0.350240,0,0,1,0
13,-1.346161,1.033250,1.858400,-0.887851,-0.822717,-0.970326,-0.851359,-0.628598,0,0,1,0
19,-1.351154,1.033250,1.858400,-0.515884,-0.567023,-0.657782,-0.584945,-0.665555,0,0,1,0


In [54]:
print(normed_test_data['ocean_proximity_INLAND'].dtype)

int64


In [55]:
normed_test_data = normed_test_data.values
normed_train_data = normed_train_data.values

### <a name='a4'></a> 5. Budowa modelu

In [56]:
def build_model():
    model = Sequential()
    model.add(Dense(1024, kernel_regularizer='l2', activation='relu', input_shape=[len(train_dataset.keys())]))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1))

    model.compile(optimizer='adam',
                  loss='mse',
                  metrics=['mae', 'mse'])
    return model

In [57]:
model = build_model()
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 1024)           │        13,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 603,905 (2.30 MB)

 Trainable params: 603,905 (2.30 MB)

 Non-trainable params: 0 (0.00 B)

### <a name='a5'></a> 6. Trenowanie sieci

In [58]:
history = model.fit(normed_train_data, train_labels.values, epochs=150, validation_split=0.2, verbose=1, batch_size=32)

Epoch 1/150
409/409 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - loss: 36970090496.0000 - mae: 153015.7031 - mse: 36970090496.0000 - val_loss: 6372820480.0000 - val_mae: 58217.6055 - val_mse: 6372820480.0000
Epoch 2/150
409/409 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - loss: 5990871552.0000 - mae: 56017.2891 - mse: 5990871552.0000 - val_loss: 4611885568.0000 - val_mae: 48816.4492 - val_mse: 4611885568.0000
Epoch 3/150
409/409 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - loss: 4675111424.0000 - mae: 48896.0859 - mse: 4675111424.0000 - val_loss: 4324369920.0000 - val_mae: 46805.7656 - val_mse: 4324369920.0000
Epoch 4/150
409/409 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - loss: 4419607040.0000 - mae: 47472.1172 - mse: 4419607040.0000 - val_loss: 4303235584.0000 - val_mae: 46889.5508 - val_mse: 4303235584.0000
Epoch 5/150
409/409 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - loss: 4363994112.0000 - mae: 46892.5195 - mse: 4363994112.0000 - val_loss: 4243800320.0000 - val_mae: 46691.7578 - val_mse: 4243800320.0000
Epoch 6/150
40

In [59]:
def plot_hist(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    hist['rmse'] = np.sqrt(hist['mse'])
    hist['val_rmse'] = np.sqrt(hist['val_mse'])

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['mae'], name='mae', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_mae'], name='val_mae', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='MAE vs. VAL_MAE', xaxis_title='Epoki', yaxis_title='Mean Absolute Error', yaxis_type='log')
    fig.show()

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['rmse'], name='rmse', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_rmse'], name='val_rmse', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='RMSE vs. VAL_RMSE', xaxis_title='Epoki', yaxis_title='Root Mean Squared Error', yaxis_type='log')
    fig.show()

plot_hist(history)

In [60]:
test_predictions = model.predict(normed_test_data).flatten()
test_predictions

128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


array([244901.05 , 203906.16 , 164435.47 , ..., 122822.625, 102642.57 , 115323.945], dtype=float32)

In [61]:
pred = pd.DataFrame(test_labels)
pred['predictions'] = test_predictions
pred.head()

,median_house_value,predictions
10,281500.000000,244901.046875
13,191300.000000,203906.156250
19,162900.000000,164435.468750
28,108900.000000,125499.437500
40,155400.000000,155856.078125


In [62]:
fig = px.scatter(pred, 'median_house_value', 'predictions')
fig.add_trace(go.Scatter(x=[0, 500000], y=[0, 500000], mode='lines'))
fig.show()

In [63]:
pred.head()

,median_house_value,predictions
10,281500.000000,244901.046875
13,191300.000000,203906.156250
19,162900.000000,164435.468750
28,108900.000000,125499.437500
40,155400.000000,155856.078125


In [64]:
pred['error'] = pred['median_house_value'] - pred['predictions']
pred.head()

,median_house_value,predictions,error
10,281500.000000,244901.046875,36598.953125
13,191300.000000,203906.156250,-12606.156250
19,162900.000000,164435.468750,-1535.468750
28,108900.000000,125499.437500,-16599.437500
40,155400.000000,155856.078125,-456.078125


In [65]:
px.histogram(pred, 'error', marginal='rug', width=1000)